In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)

# Load Data

In [ ]:
sp_merge <- readRDS(file = '/home/jupyter/Organized_Gating_Analysis/Seurat_Objects/sp_merge_gating_celltype_updated.rds')

In [ ]:
sp_merge <- SetIdent(sp_merge, value = 'gating_celltype')
cd8_mem <- subset(sp_merge, idents = c('CD8 CM','CD8 EM1','CD8 EM2','CD8 TEMRA'))

In [ ]:
cd8_mem$pure_celltype <- cd8_mem$gating_celltype

In [ ]:
cd8na <- readRDS(file = 'trim_cd8_na.rds')

In [ ]:
cd8_merge <- merge(cd8na, cd8_mem)

In [ ]:
rm(sp_merge)
rm(cd8_mem)
rm(cd8na)

# Re-normalization

In [ ]:
DefaultAssay(cd8_merge) <- 'RNA'
cd8_merge <- NormalizeData(cd8_merge) %>% FindVariableFeatures() %>% ScaleData() %>% RunPCA()

In [ ]:
ElbowPlot(cd8_merge, ndims = 50)

In [ ]:
cd8_merge <- RunUMAP(cd8_merge, dims = 1:25)

In [ ]:
DimPlot(cd8_merge, group.by = 'pure_celltype')

# DEG Analysis

In [ ]:
cd8_merge <- SetIdent(cd8_merge, value = 'pure_celltype')
markers <- FindAllMarkers(cd8_merge)

In [ ]:
cd8_downsample <- subset(cd8_merge, downsample = 100)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
markers %>%
    group_by(cluster) %>%
    top_n(n = 10, wt = avg_log2FC) -> top10
DoHeatmap(cd8_downsample, features = top10$gene) + NoLegend()

# Average Expression of Markers

In [ ]:
degs_avg_exp <- AverageExpression(cd8_merge, assays = 'RNA', features = unique(markers$gene), group.by = 'pure_celltype')
degs_avg_exp <- as.data.frame(degs_avg_exp)

In [ ]:
head(degs_avg_exp)

In [ ]:
write.csv(markers, file = 'cd8_rna_degs.csv')

In [ ]:
markers <- read.csv(file='cd8_rna_degs.csv')

In [ ]:
length(unique(markers$gene))

In [ ]:
# colnames(degs_avg_exp) <- c()
# head(degs_avg_exp)

In [ ]:
# order_degs_avg_exp <- data.frame('State 2' = degs_avg_exp['State 2'],
#                                  'State 4' = degs_avg_exp['State 4'],
#                                  'State 3' = degs_avg_exp['State 3'],
#                                  'State 1' = degs_avg_exp['State 1'])
# head(order_degs_avg_exp)

# Pheatmap

In [ ]:
library(pheatmap)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 16)
rna_pheatmap <- pheatmap(degs_avg_exp, cluster_rows = TRUE, cluster_cols = TRUE, scale = 'row', cellwidth = 50, cellheight = 0.5, show_rownames = FALSE, border_color = NA,
                         # annotation_row = cell_state_genes, annotation_colors = anno_colors,
                         color = colorRampPalette(c('#67a9cf','white','#d6604d'))(100))

In [ ]:
save_pheatmap_pdf <- function(x, filename, width=10, height=16) {
   stopifnot(!missing(x))
   stopifnot(!missing(filename))
   pdf(filename, width=width, height=height)
   grid::grid.newpage()
   grid::grid.draw(x$gtable)
   dev.off()
}
save_pheatmap_pdf(rna_pheatmap, "plots/CD8_global_RNA_heatmap.pdf")